# Praktische Optimierung Blatt 07
### Tobias Lotz:  217856 <br>
### Alexander van der Staay:  185444

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from scipy.optimize import minimize

### Aufgabe 7.1

$f(x,y) = x \cdot y$, $\space \space$ mit Nebenbedingung $\space$ $g: x - y = 3$ <br> 

$x - y = 3$ $\space \space \space$| $+ y$ <br>
$x = 3 + y$ <br>

In $f$ einsetzen: <br> <br>
$f(y) = y \cdot (y + 3)$ <br>

Erste Ableitung gleich 0 setzen: <br> <br>
$f^{'}(y) = 2y + 3$ <br>

$0 = 2y + 3 \space \Rightarrow \space y = -1.5$ <br>

Zweite Ableitung überprüfen: <br> <br>
$f^{''}(y) = 2 > 0 \space \Rightarrow \space y = -1.5$ ist lokales Minimum von $f$. <br>

$y = -1.5$ einsetzen um $x$ zu bestimmen: <br> <br>
$x = 3 + -1.5 \space \Rightarrow \space x = 1.5$ <br>

Also ist das bestimmte Minimum für $f$ mit Nebenbedingung $g: \space \space$ $(1.5, -1.5)$ 

In [2]:
def f(x): return x[0] * x[1]

In [3]:
x = np.round(np.arange(-10, 10, 0.1), decimals=2)
y = np.round(np.arange(-10, 10, 0.1), decimals=2)
X, Y = np.meshgrid(x, y)
Z = X * Y

surfacecolor = np.zeros_like(Z)
xs = []
ys = []
zs = []
for i in range(X.shape[0]):
    for j in range(X[0].shape[0]):  
        if X[i][j] - Y[i][j] == 3:
            xs.append(X[i][j])
            ys.append(Y[i][j])
            zs.append(X[i][j] * Y[i][j])
            surfacecolor[i][j] = 1

fig = go.Figure(data=[go.Surface(z=Z,
                                 x=X, 
                                 y=Y,
                                 surfacecolor=surfacecolor)])
fig.add_trace(go.Scatter3d(x=xs, y=ys, z=zs, mode="lines", line_width=7, line_color="red"))
fig.add_trace(go.Scatter3d(x=[1.5], y=[-1.5], z=[f([1.5, -1.5])], mode='markers', name="Minimum"))
fig.update_layout(title='3D-Plot der Funktion f', autosize=False,
                  width=500, height=500,
                  margin=dict(l=65, r=50, b=65, t=90))
fig.show()

Die rote Linie stellt alle Gültigen Werte der Nebenbedingung dar. Der grüne Punkt ist das bestimmte Minimum

### Aufgabe 7.2

In [4]:
LOWER = -10
UPPER = 10

In [5]:
# (1 + lambda)- bzw. (1, lambda)-EA:
def one_lambda_ea(f, lambda_val, parent,lower, upper, repair_method, method="plus", sigma0=0.1, tau=0.1, evals=500):
   
    # initialize values:
    fitness_p = f(parent)
    evals -= 1

    while evals > 0:
        # schwefel method:
        sigma0 *= np.exp(np.random.normal(loc=0, scale=tau))
        lambda_val = min(lambda_val, evals)
        children = parent + sigma0 * np.random.normal(size=(lambda_val, len(parent)))

        #repair invalid children
        if repair_method == 'Rand':
            for child in children:
                for i, val in enumerate(child):
                    if val < lower:
                        child[i] = lower
                    elif val > upper:
                        child[i] = upper
        
        elif repair_method == 'Modulo':
            for child in children:
                for i, val in enumerate(child):
                    if val < lower:
                        child[i] = (upper - (lower - val)) % (upper - lower)
                    elif val > upper:
                        child[i] = (lower + (val - upper)) % (upper - lower)

        
        else: # Achsenspiegelung
            for child in children:
                for i, val in enumerate(child):
                    if val < lower:
                        child[i] = lower + (lower - val)
                    elif val > upper:
                        child[i] = upper - (val - upper)
    
        #evaluate children:
        fitness_c = np.apply_along_axis(f, 1, children)
        evals -= lambda_val

        # selection for next generation:
        if method == "plus":
            population = np.concatenate(
                (np.expand_dims(parent, axis=0), children))
            fitness_all = np.concatenate(
                (np.array([fitness_p]), fitness_c))
            min_ind = np.argmin(fitness_all)
            parent = population[min_ind]
            fitness_p = fitness_all[min_ind]
        else:
            min_ind = np.argmin(fitness_c)
            parent = children[min_ind]
            fitness_p = fitness_c[min_ind]

    # return best values found:
    return {'x': parent, 'fx': fitness_p}

In [6]:
def f(x): return x[0]**3 * np.sin(x[0] - 1) - x[1]**3 * np.cos([x[1]])

In [7]:
# ---------- Rand --------------------------
np.random.seed(1)
results_rand = []
best_points_rand = []
for _ in range(1000):
    res = one_lambda_ea(f, 5, [0,0], LOWER, UPPER, "Rand", sigma0=5, tau=0.1, evals=100)
    results_rand.append(res['fx'])
    best_points_rand.append(res['x'])

In [8]:
# ---------- Modulo --------------------------
np.random.seed(1)
results_mod = []
best_points_mod = []
for _ in range(1000):
    res = one_lambda_ea(f, 5, [0,0], LOWER, UPPER, "Modulo", sigma0=5, tau=0.1, evals=100)
    results_mod.append(res['fx'])
    best_points_mod.append(res['x'])

In [9]:
# ---------- Achsenspiegelung --------------------------
np.random.seed(1)
results_achse = []
best_points_achse = []
for _ in range(1000):
    res = one_lambda_ea(f, 5, [0,0], -10, 10, "Achsenspiegelung", sigma0=5, tau=0.1, evals=100)
    results_achse.append(res['fx'])
    best_points_achse.append(res['x'])

In [10]:
results_rand = np.array(results_rand)
results_mod = np.array(results_mod)
results_achse = np.array(results_achse)

print(f'Mittlerer Funktionswert:\nRand\t\tModulo\t\tAchsenspiegelung')
print(f'{results_rand.mean() : .2f}\t{results_mod.mean() : .2f}\t{results_achse.mean() : .2f}')

Mittlerer Funktionswert:
Rand		Modulo		Achsenspiegelung
-1453.88	-7542.80	-1598.86


Man sieht, dass die Lösungen des Rand-Mechanismus häufig genau den Rand-Wert in einer oder beiden Komponenten annehmen

In [11]:
np.array(best_points_rand)[:10]

array([[-10.        ,   6.69781075],
       [-10.        ,  -9.95922402],
       [  6.74407425,   6.31023375],
       [-10.        ,  -9.80921163],
       [-10.        ,   6.60561659],
       [-10.        ,   6.35502886],
       [-10.        , -10.        ],
       [-10.        ,  -9.6178701 ],
       [-10.        ,   6.59189806],
       [-10.        ,  -9.71788177]])

Der Modulo Reparaturmechanismus scheint am besten zu funktionieren. Ein Problem des Rand-Mechanismus ist, dass es sehr wahrscheinlich ist, dass neu generierte Kinder wieder außerhalb des Intervalls liegen. Wenn ein Kind außerhalb des Intervalls liegt, wird dieses lediglich genau auf den Rand gesetzt. Wenn dieses Kind nun als Parent ausgewählt wird, werden neue Kinder in der Umgebung des Randes generiert. Es ist daher wahrscheinlich das die finale Lösung am Rand des Intervalls liegt. Dies ist zu erkennen, wenn man sich die Lösungen des Algorithmus anguckt.  

### Aufgabe 7.3

In [24]:
def f(x): return (x[0] - 2)**2 + (x[1] - 1)**2

def g1(x): return x[0]**2 - 2 * x[0] + x[1]

# +2 damit der ganze Term <= 0 sein muss
def g2(x): return -x[0] + x[1] + 2

In [38]:
def default_phi(t): return max(0, t)

In [45]:
def sequentielle_straffung(f : callable, x0 : np.ndarray, g1 : callable, g2 : callable, 
                           xi_0 : float, gamma : float, phi=default_phi, max_iter = 500000):
    found_valid_solution = False
    x_curr = x0
    xi = xi_0
    it = 0
    while not found_valid_solution and it < max_iter:
        def t(x): return f(x) + xi * (phi(g1(x)) + phi(g2(x)))
        res = minimize(fun=t, x0=x_curr, method='BFGS')
        possible_solution = res['x']
        if g1(possible_solution) <= 0 and g2(possible_solution) <= 0:
            return {"x" : possible_solution, "fx" : f(possible_solution)}
        
        xi *= gamma
        it += 1

In [55]:
xi_init = np.arange(0.1, 1.1, 0.1)
gamma_init = np.arange(1.1, 2.1, 0.1)

In [65]:
np.random.seed(1)

average_funcval = np.zeros((10, 10))

for i, xi in enumerate(xi_init):
    for j, gamma in enumerate(gamma_init):
        x0_init = np.random.uniform(-10, 10, size=(20, 2))
        func_vals = []
        for x0 in x0_init:
            func_vals.append(sequentielle_straffung(f, x0, g1, g2, xi, gamma)['fx'])
        average_funcval[i][j] = np.array(func_vals).mean()

In [66]:
average_funcval

array([[1.00522876, 1.01278744, 1.02127332, 1.00544816, 1.01897037,
        1.08492604, 1.0111122 , 1.02613694, 1.0103788 , 1.01589501],
       [1.01233385, 1.0231507 , 1.00606966, 1.01139172, 1.0023358 ,
        1.01766629, 1.02293117, 1.01006714, 1.01168985, 1.01485505],
       [1.00238909, 1.00885444, 1.0086669 , 1.01288235, 1.00868705,
        1.02309253, 1.01017217, 1.01121211, 1.00765023, 1.00373209],
       [1.00519262, 1.00448997, 1.01524155, 1.00705435, 1.01178777,
        1.00922397, 1.02666885, 1.01464826, 1.01029607, 1.05765444],
       [1.0222756 , 1.00248762, 1.00896131, 1.01671493, 1.00452492,
        1.00999576, 1.01339167, 1.01135583, 1.33474393, 1.03432037],
       [1.00811848, 1.01125777, 1.01413321, 1.01392623, 1.00822703,
        1.01815781, 1.00840518, 1.00818954, 1.02234999, 1.01769047],
       [1.01011236, 1.00868116, 1.01158954, 1.01240184, 1.0049195 ,
        1.01190268, 1.02077447, 1.03535615, 1.02209018, 1.04390658],
       [1.00202943, 1.02446492, 1.0129257

In [63]:
x0_init = np.random.uniform(-10, 10, size=(20, 2))
func_vals = []
for x0 in x0_init:
    func_vals.append(sequentielle_straffung(f, x0, g1, g2, 0.5, 1.5)['fx'])

In [64]:
func_vals

[1.0000217407822438,
 1.0000079764404626,
 1.0308153251254408,
 1.0021233229243227,
 1.0013555344789622,
 1.0013333729750251,
 1.0378297649434114,
 1.0202310743706087,
 1.0053576572798204,
 1.020849397712205,
 1.0000317352767933,
 1.0000000247843552,
 1.0003622697326724,
 1.0013994984365042,
 1.0026884391457764,
 1.001711344064059,
 1.0000107870495143,
 1.0000000031859213,
 1.0000949722356842,
 1.0016137752774894]